In [2]:
import networkx as nx
import numpy as np

In [10]:
def nb_samples_required(a, delta, epsilon):
    return int(np.ceil(2 * (np.log(2) * a + np.log(1 / delta)) / (epsilon**2)))

In [21]:
# Create all graphlets of size 3, 4, 5
def build_graphlets(k):
    if k == 3:
        graphlets = [
            np.zeros((3, 3)),
            np.array([[0, 1, 0], [1, 0, 0], [0, 0, 0]]),
            np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]]),
            np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]])
        ]
        return [nx.from_numpy_matrix(m) for m in graphlets]

In [5]:
def sample_subgraphs(G, k):
    nodes = G.nodes()

    return G.subgraph(np.random.choice(nodes, k, replace=False).tolist())

In [25]:
def find_iso_graphlet(subgraph, graphlets, spectrum):
    for i, graphlet in enumerate(graphlets):
        if nx.is_isomorphic(subgraph, graphlet):
            spectrum[i] += 1

In [30]:
def estimate_spectrum(G, k, delta, epsilon):
    graphlets = build_graphlets(k)
    m = nb_samples_required(len(graphlets), delta, epsilon)
    spectrum = np.zeros(len(graphlets))

    for _ in range(m):
        subgraph = sample_subgraphs(G, k)
        find_iso_graphlet(subgraph, graphlets, spectrum)

    #return spectrum / np.linalg.norm(spectrum) # to sum to 1 for kernel(G, G)
    return spectrum / m

In [8]:
def kernel(G1, G2, k, delta, epsilon):
    spectrum1 = estimate_spectrum(G1, k, delta, epsilon)
    spectrum2 = estimate_spectrum(G2, k, delta, epsilon)

    return spectrum1.T @ spectrum2

In [31]:
G = nx.fast_gnp_random_graph(100, 0.1)

spectrum = estimate_spectrum(G, 3, 0.05, 0.05)

In [32]:
spectrum

array([0.94863033, 0.3150888 , 0.02859347, 0.00140164])

In [33]:
kernel(G, G, 3, 0.05, 0.05)

0.9997956917241628